In [ ]:
import tweepy
import cnfg
from tqdm import tqdm
from pymongo import MongoClient
import numpy as np
import json

In [ ]:
###  SET UP AUTHENTICATION  ###

auth = tweepy.OAuthHandler('API KEY',
                           'REMOVED')
auth.set_access_token('TO PREVENT',
                      'GITHUB ABUSE')

In [ ]:
api=tweepy.API(auth)
username = 'elonmusk'
tweets = tweepy.Cursor(api.user_timeline, screen_name=username, tweet_mode='extended').items()

In [ ]:
###  CREATE LIST OF DICTIONARIES WHERE EACH TWEET HAS THE FOLLOWING FIELDS  ###

tweet_dict_list = []

for t in tqdm(tweets):
    d = {}
    d['created_at'] = t.created_at
    d['favorite_count'] = t.favorite_count
    d['retweet_count'] = t.retweet_count
    d['full_text'] = t.full_text
    d['screen_name'] = t.user.screen_name

    tweet_dict_list.append(d)

In [ ]:
###  ENGINEERING VIRALITY MEASURE  ###

full_tweets = []
for item in tweet_dict_list:
    full_tweets.append(item['full_text'].lower().replace('\n', ' '))
    item['virality'] = item['favorite_count'] + 10*item['retweet_count']

In [ ]:
tweet_dict_list[7]

In [ ]:
full_tweets

In [ ]:
###  CLEANING TWEET TEXT  ###

cleaned_tweets = []
for tweet in full_tweets:
    storage = []
    for word in tweet.split():
        if len(word) <3:
            continue
        if '@' in word:
            continue
        if 'https://' in word:
            continue
        storage.append(word)
    cleaned_tweets.append(' '.join(storage))


for i,tweet in enumerate(tweet_dict_list):
    tweet['cleaned_text'] = cleaned_tweets[i]

In [ ]:
###  Storing Tweets in MongoDB  ###

client = MongoClient('localhost', 27017)
client.list_database_names()

In [ ]:
elon_db = client['elon_db']
tweets = elon_db.tweets
tweets.insert_many(tweet_dict_list)
tweets.count()

In [ ]:
###  TESTING BY PRINTING MOST VIRAL TWEETS  ###

for tweet in tweets.find({'virality' : {'$gt':800000}}):
    print(tweet['cleaned_text'])